# Decoding .FIT files into Pandas dataframes and .CSV's
This script decodes the 'record' frames in FIT files into Pandas DataFrames using the fitdecode library. You easily can convert Pandas dataframes into various common file types, including CSV.

In [96]:
import fitdecode
import pandas as pd

# Let's create a useful function for decoding FIT files

def DecodeFit_DataFrame(file_path: str, frame_name: str = 'record', lat_long_update: bool = True, debug: bool = False) -> pd.DataFrame:
    # Initialize some useful variables for the loops
    check_list = good_list = []
    list_check = {}
    df_activity = pd.DataFrame([])

    # Open the file with fitdecode
    with fitdecode.FitReader(file_path) as file:
        
        # Iterate through the .FIT frames
        for frame in file:
            
            # Procede if the frame object is the correct data type
            if isinstance(frame, fitdecode.records.FitDataMessage):
                
                # Add the frames and their corresponding counts to a dictionary for debugging
                if frame.name not in check_list:
                    check_list.append(frame.name)
                    list_check[frame.name] = 1
                else:
                    list_check.update({frame.name: list_check.get(frame.name) + 1})
                
                # If the current frame is a record, we'll reset the row_dict variable
                # and add the field values for all fields in the good_list variable
                if frame.name == frame_name:
                    row_dict = {}
                    for field in frame.fields: 
                        if field.name.find('unknown') < 0:
                            if field.name not in good_list and field.name.find('unknown') < 0:
                                good_list.append(field.name)
                            row_dict[field.name] = frame.get_value(field.name)
                    
                    # Append this row's dictionary to the main dataframe
                    df_activity = pd.concat([df_activity, pd.DataFrame([row_dict])], ignore_index = True)
        
        # Update the Long/Lat columns to standard degrees
        if lat_long_update:
            for column in ['position_lat', 'position_long']:
                df_activity[column] = df_activity[column].apply(lambda x: x / ((2**32)/360))
        
        # If you want to check to see which frames are in the file, print the list_check variable
        if debug:
            print(list_check)

    return df_activity

df_activity = DecodeFit_DataFrame('[YOUR FILE PATH HERE]', frame_name = 'record')

## Export Options

With the df_activity DataFrame you can export the FIT information to other file types, including CSVs.

For a full list of options visit the [Pandas DataFrame API documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_csv.html).

In [64]:
# To CSV
df_activity.to_csv('Decoded_FIT_File.csv')

# To Clipboard
df_activity.to_clipboard(sep = ',', index = False)

df_activity.head()

,timestamp,position_lat,position_long,distance,enhanced_speed,enhanced_altitude,vertical_oscillation,stance_time_percent,stance_time,vertical_ratio,stance_time_balance,step_length,heart_rate,cadence,temperature,activity_type,fractional_cadence
0,2022-01-25 23:51:50+00:00,41.740804,-111.827001,0.00,1.847,1391.2,96.5,35.00,257.0,14.62,47.03,552.0,104,81,26,running,0.5
1,2022-01-25 23:51:51+00:00,41.740815,-111.826965,3.33,2.650,1391.2,96.5,36.00,259.0,14.25,47.28,978.0,109,83,26,running,0.5
2,2022-01-25 23:51:53+00:00,41.740832,-111.826893,10.36,3.406,1391.2,99.5,36.75,257.0,10.53,47.62,1212.0,113,85,25,running,0.5
3,2022-01-25 23:51:56+00:00,41.740831,-111.826782,19.94,3.014,1391.4,83.0,36.50,256.0,8.00,48.34,1179.0,115,85,25,running,0.5
4,2022-01-25 23:51:57+00:00,41.740826,-111.826745,23.30,3.182,1392.0,90.7,36.25,255.0,7.90,48.46,1160.0,120,85,25,running,0.0


## Other useful variables

You can add useful variables to your DataFrame, like 'elapsed time' or 'standardized altitude' .

In [74]:
# How much time has passed since the beginning of the activity?
df_activity['elapsed_time'] = df_activity['timestamp'].apply(lambda x: x - df_activity.loc[0, 'timestamp'])

# Standardize altitude around 0, from 1 to -1
df_activity['standardized_altitude'] = ((df_activity['enhanced_altitude'] - df_activity['enhanced_altitude'].min()) / (df_activity['enhanced_altitude'].max() - df_activity['enhanced_altitude'].min()) * 2) - 1

## Combine through multiple .FIT files
Import 'os' to grab FIT files in a directory, then loop through them and concatenate the resulting DataFrames together

In [97]:
import os

# Select a directory with FIT files in it
os.chdir('[YOUR DIRECTORY PATH HERE]')

# Generate the list of file paths
fit_files_list = os.listdir()

# Set up a 'Master' DataFrame to add all the fit files to
df_activity_combined = pd.DataFrame([])

# Loop through the list of fiel paths and add each Activity's DataFrame to the df_activitty_combined DataFrame
for file in fit_files_list:
    # Call DecodeFit_DataFrame
    df_interim = DecodeFit_DataFrame(file)

    # Need help remembering which file this data is from?
    df_interim['file'] = file

    # Use pd.concat!
    df_activity_combined = pd.concat([df_activity_combined, df_interim], ignore_index = True)

# Take a look at the continuous variables
df_activity_combined.describe()

,position_lat,position_long,distance,enhanced_speed,enhanced_altitude,vertical_oscillation,stance_time_percent,stance_time,vertical_ratio,stance_time_balance,step_length,heart_rate,cadence,fractional_cadence,altitude,power
count,10513.000000,10513.000000,10513.000000,10513.000000,10513.000000,1295.000000,1293.000000,1293.000000,1295.000000,1293.000000,1295.000000,10513.000000,10513.000000,1295.000000,9218.000000,9218.000000
mean,-5.092028,132.612111,14775.579420,6.629419,267.700143,88.740232,36.846288,256.456303,7.257568,48.683163,1196.138224,151.590222,79.148007,0.246332,109.668583,179.618898
std,17.556107,91.623435,9958.778999,3.702979,441.272513,4.348962,0.950756,7.620994,0.841243,0.446777,98.399464,14.903861,9.981155,0.250070,138.959434,52.091423
min,-11.692996,-111.829482,0.000000,0.000000,-6.200000,73.000000,34.250000,234.000000,6.030000,47.030000,552.000000,74.000000,0.000000,0.000000,-6.200000,0.000000
25%,-11.683089,166.930136,6231.580000,3.761000,19.600000,85.700000,36.250000,251.000000,6.710000,48.370000,1141.500000,142.000000,76.000000,0.000000,16.800000,130.000000
50%,-11.677088,166.950721,14204.370000,5.758000,57.200000,87.700000,37.000000,256.000000,7.120000,48.650000,1212.000000,155.000000,81.000000,0.000000,48.300000,190.000000
75%,-11.647672,166.969088,21066.130000,9.020000,315.400000,90.700000,37.500000,261.000000,7.590000,48.960000,1265.000000,162.000000,86.000000,0.500000,152.150000,215.000000
max,41.750017,166.980654,38855.720000,23.523000,1408.000000,111.200000,40.250000,293.000000,14.620000,50.280000,1434.000000,188.000000,100.000000,0.500000,516.200000,571.000000
